In [ ]:
import pandas as pd
import dateutil
import pypsa
import pickle
import re
import numpy as np
from _helpers import configure_logging

import logging
logger = logging.getLogger(__name__)

if __name__ == "__main__":
    
    scenario = snakemake.params["scenario"]
    configure_logging(snakemake)
    
    with open(snakemake.input["additional_components"], "rb") as f:
        override_component_attrs = pickle.load(f)
        
    network = pypsa.Network(override_component_attrs=override_component_attrs)
    network.import_from_netcdf(snakemake.input["network"])

    # Read load profile from file
    df = pd.read_csv(snakemake.input["import_profile"])
    
    if {"snapshot","profile [p.u.]"} != set(pd.read_csv(snakemake.input["import_profile"]).columns):
        logger.error(f"Import profile file '{snakemake.input['import_profile']}' must have "
                     f"exactly two columns 'snapshot' and 'profile [p.u.]'")
    
    if np.any(pd.to_datetime(df["snapshot"]) != network.snapshots):
        logger.error(f"At least one 'snapshot' entry in '{snakemake.input['import_profile']}' "
                     f"does not match the network snapshots: {network.snapshots[:5], ...}.")
    
    df = df.set_index("snapshot")

    # Apply the import_profile to all loads in the network
    for load, load_s in network.loads.iterrows():
        import_profile = (load_s["p_set"]*df["profile [p.u.]"])
        network.loads_t["p_set"][load] = import_profile
        network.loads.loc[load, "p_set"] = 0

    network.export_to_netcdf(snakemake.output['network'])